In [3]:
import tarfile
import os
from loguru import logger

In [12]:

tarfile.open()

AttributeError: module 'loguru' has no attribute 'Logger'

In [40]:
"""path"""

WORK_DIR = os.path.curdir
DEBUG_DIR = os.path.join(WORK_DIR,'debug')
# @TODO argparse data path


"""FOR DEBUG"""
DATA_ROOT = os.path.join(DEBUG_DIR,'example')
TMP_DIR = os.path.join(DEBUG_DIR,'tmp')
if not os.path.isdir(TMP_DIR):
    os.makedirs(TMP_DIR)
OUT_DIR = os.path.join(DEBUG_DIR,'output')


In [17]:
"""logging"""

DEBUG = True

LOG_DIR = os.path.join(WORK_DIR,'log')
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

LOG_FILE = os.path.join(LOG_DIR,'debug_{time}.log')

logger.add(LOG_FILE,retention=2)

if DEBUG:
    logger.debug('debug data directory: {}'.format(DEBUG_DIR))

2022-06-24 18:33:28.137 | INFO     | __main__:<module>:12 - log saved in .\log\debug_{time}.log
2022-06-24 18:33:28.138 | DEBUG    | __main__:<module>:15 - debug data directory: .\debug


import tarfile

dir_1 = './debug/example/AA/'
tar_1 = os.path.join(dir_1, 'AABAAC.tar')
dir_2 = os.path.join(TMP_DIR, 'AABAAC')
tar_2_gz =  os.path.join(dir_2, '00000.tar.gz')

with  tarfile.open(tar_1) as tar:
    tar.extractall(TMP_DIR)
    names = tar.getnames()


In [43]:
from tqdm import tqdm

for d in os.listdir(DATA_ROOT):
    d_path = os.path.join(DATA_ROOT,d)
    for f in tqdm(os.listdir(d_path)):
        # if i == 2: break
        with tarfile.open(os.path.join(d_path,f)) as tf:
            names = tf.getnames()
            tf.extractall(TMP_DIR)
        for name in names:
            dirname = os.path.dirname(name)
            with tarfile.open(os.path.join(TMP_DIR, name)) as tf:
                tf.extractall(os.path.join(TMP_DIR, dirname))

 42%|████▏     | 86/204 [00:09<00:12,  9.44it/s]


KeyboardInterrupt: 

In [30]:
os.path.dirname(names[0])

'AABAAC'